#### 1.

#### 2. Missing value

In [ ]:
'''
결측치 처리법 3가지
1. 결측행 제거
2. 결측값 대체
3. 결측값 대체 및 대체유무에 대해 기입

2번과 3번에서 "대체"시에 SimpleImputer를 활용
Kaggle: SimpleImputer는 평균값으로 값을 대체. 사람들은 보통 결측값을 complex한 방법으로 대체하기 위해
다양한 방법을 활용 (ex. regression imputation). 그러나 이는 복잡한 ML모델이 활용된다면,
이에 대한 이점은 거의 없어진다고 보면 된다.

==============================================================================================================================

궁금: fit_transform과 transform의 차이는 무엇인가?

fit_transform() 메소드는 말 그대로 fit()한 다음에 transform() 하는 것입니다. 
여기서 fit()이란 정규화 즉, 통계에서 정규분포를 만들게 하기 위해서 𝑥 값에서 평균을 빼고 
그 값을 다시 표준편차로 나누어주는 작업을 하는데 
이 작업을 하기 위해 평균 𝜇과  표준편차 𝜎를 계산하는 작업이 fit() 이고, transform()은 정규화 작업을 해주는 것입니다. 
(𝑥-𝜇)/𝜎 ==> 새로운 𝑥′ 생기는 것이죠

트레이닝 데이터에 대해서 fit 작업과 transform 작업을 적용해주는 것이 fit_transform이고 
여기서 계산된 평균 𝜇과  표준편차 𝜎를 동일하게 테스트 데이터에 적용해서 정규화 작업을 해주는 경우는 transform()만 적용합니다.  
테스트 세트에도 트레이닝 데이터에 적용되었던 동일한 평균과 표준편차를 적용하기 위해서 
이전에 fit_transform()에서 계산된 값들이 저장된 상태에서 transform()에 적용되는 것입니다.
아래 출처를 따라가 보시면 fit_transform의 파라미터들을 자세히 살펴보실 수 있습니다.
'''

In [ ]:
# 특정 열에서 결측치가 있는 행을 제거할 때 subset 인자를 활용
X_full.dropna(axis=0, subset=['LotFrontage'], inplace=True)

# 자료형이 object가 아닌 것만 불러오기
X = X_full.select_dtypes(exclude=['object'])

# SimpleImputer에 평균 대신 중간값으로 대체할 수 있는 방법
final_imputer = SimpleImputer(strategy='median')

In [ ]:
'''
열 별 결측치의 개수를 보고 해당 열을 제거해야하는가? 생각
--> 열 자체를 제거하는 것은 귀중한 데이터를 제거하는 것이다. 하지만, 결측치의 개수가 열 전체의 개수의 20%미만이라면 제거하지말자.
오히려 이럴땐 대체하는 것이 바람직 할 수 있다.

====================================================================================================================

데이터 세트에 결측값이 너무 적기 때문에 열을 완전히 삭제하는 것보다 귀책성이 더 잘 수행될 것으로 예상한다. 
그러나 열을 떨어뜨리는 것이 약간 더 낫다는 것을 알 수 있습니다. 
이는 데이터 세트의 노이즈에 부분적으로 기인할 수 있지만, 
또 다른 잠재적인 설명은 귀책 방법이 이 데이터 세트와 크게 일치하지 않는다는 것이다.

즉, 평균 값을 채우는 대신 모든 결측값을 0 값으로 설정하거나, 
가장 자주 발생하는 값을 채우거나, 다른 방법을 사용하는 것이 더 합리적일 수 있습니다.

각 열을 따라 중위수 값을 채우는 것이 더 의미가 있습니까? 
아니면 각 열을 따라 최소값을 채워 더 나은 결과를 얻을 수 있을까요? 
이 경우에 무엇이 최선인지 확실하지 않지만, 아마도 우리는 즉시 몇 가지 옵션을 배제할 수 있습니다. 
    예를 들어, 이 열의 결측값을 0으로 설정하면 끔찍한 결과를 초래할 수 있습니다!
'''

#### 3. Categorical Values

In [ ]:
'''
카테고리 변수 CITY처럼 경기, 강원을 value로 갖는 변수의 경우 처리방법
1. Drop
2. 숫자에 mapping
 - sklearn의 labelencoder를 쓰면 편리 --> 그러나, 각 값에 무작위 숫자를 대응
 - 이렇게 되면 크기요소를 반영하기 힘들듯 싶은데?
3. one-hot-encoding
 - sklearn의 OneHotEncoder를 쓴다.
 - 파라미터 설명
  -- handle_unknown='ignore': train에 원-핫-인코딩 후, validation data에서 없는 카테고리 발견시 오류발생x 
  -- sparse=False: 반환객체가 numpy
  
*** labelencoder나 onehotencoder 활용 전에 꼭 train과 valid data의 구성이 맞는지 확인해!
*** onehotencoding은 변수의 value수가 적을때 많이 활용되고, nominal이지만 value의 수가 많다면 아싸리 제거하거나 labelencoding
'''

궁금! : handle_unknown='ignore'을 실시시, 없는 값은 그냥 제외되는 것인지?

In [ ]:
# map과 lambda함수

# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols)) #map과 lambda는 항상 같이 다님. map(lambda 인자: 함수식, 넣을 인자)
d = dict(zip(object_cols, object_nunique)) # [(, ), (, ), (, ) ]등의 형태로 다수 들어가며, dict함수를 거침으로써 dictionary로 변환

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

#### 4. pipline

In [ ]:
# 파이프라인 구성법1

import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X_full = pd.read_csv('../input/train.csv', index_col='Id')
X_test_full = pd.read_csv('../input/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

In [ ]:
# 파이프라인 구성법2

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)

print('MAE:', mean_absolute_error(y_valid, preds))

#### 5. Cross validation

In [ ]:
'''
데이터가 적을때, CV를 쓰자. 데이터가 적은데, Validation set까지 빼내려먼 훈련data가 적어지기 떄문.
데이터가 크면, single validation을 실시하자! (크면 시간이 많이 걸리잖아!)
Pipline을 활용하면 CV가 훨씬 쉬움!
'''

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

my_pipeline = Pipeline(steps=[('preprocessor', SimpleImputer()),
                              ('model', RandomForestRegressor(n_estimators=50,
                                                              random_state=0))
                             ])

In [ ]:
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)

In [ ]:
print("Average MAE score (across experiments):")
print(scores.mean())